In [1]:
import torch, torchvision
import sys # Python system library needed to load custom functions
import math # module with access to mathematical functions
import os # for changing the directory

import numpy as np  # for performing calculations on numerical arrays
import pandas as pd  # home of the DataFrame construct, _the_ most important object for Data Science
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt  # allows creation of insightful plots

sys.path.append('../../audio_preprocessing')
sys.path.append('../../src')
sys.path.append('../../model_training_utils')


import preprocessing_func_3
from generator_to_dataset_3 import NormalisedDataSet
from gdsc_utils import PROJECT_DIR
import model_training
import model_eval

os.chdir(PROJECT_DIR) # changing our directory to root

In [2]:
df_big_data = pd.read_csv('data/big_processed_data.csv')
df_big_argumented_data = pd.read_csv('data/big_argumentation_data.csv')
df = pd.concat([df_big_data, df_big_argumented_data], ignore_index=True)
df.tail()

,Unnamed: 0,file_path,label
66835,44959,data/big_data_upsample/44959.wav,65
66836,44960,data/big_data_upsample/44960.wav,65
66837,44961,data/big_data_upsample/44961.wav,65
66838,44962,data/big_data_upsample/44962.wav,65
66839,44963,data/big_data_upsample/44963.wav,65


In [3]:
import json

with open('audio_preprocessing/saved_data/upsampled_data_size_128_512.json') as f:
    my_info = json.load(f)

mean, std, class_weights = my_info["mean"], my_info["std"], my_info["weights"]

In [4]:
train_df_list = []
val_df_list = []

for i in range(66):
    my_df = df[df["label"] == i]
    current_train_df, current_val_df = train_test_split(my_df, test_size=0.2)
    train_df_list.append(current_train_df)
    val_df_list.append(current_val_df)

df_train = pd.concat(train_df_list, ignore_index=True)
df_val = pd.concat(val_df_list, ignore_index=True)

In [5]:
df_train.shape, df_val.shape

((53441, 3), (13399, 3))

In [6]:
train_dataset = NormalisedDataSet(
    non_normalised_data_generator_fn=preprocessing_func_3.non_normalised_data_generator, 
    normalised_data_generator_fn=preprocessing_func_3.normalised_data_generator,
    df=df_train, 
    mean=mean,
    std=std,
    shuffle=True
)

val_dataset = NormalisedDataSet(
    non_normalised_data_generator_fn=preprocessing_func_3.non_normalised_data_generator, 
    normalised_data_generator_fn=preprocessing_func_3.normalised_data_generator,
    df=df_val, 
    mean=mean,
    std=std,
    shuffle=False
)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=28)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=28)

In [7]:
device = model_training.get_device()

In [8]:
from torchvision.models import efficientnet_v2_s
import torch.optim as optim
import torch.nn as nn

#resnet_model = resnet34(weights=ResNet34_Weights.DEFAULT)
efficientnet_v2_s_model = efficientnet_v2_s(num_classes=66)
efficientnet_v2_s_model.features[0][0] = nn.Conv2d(1, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
efficientnet_v2_s_model = efficientnet_v2_s_model.to(device)

In [9]:
optimizer = optim.Adam(efficientnet_v2_s_model.parameters(), amsgrad=True)
loss = nn.CrossEntropyLoss()

In [10]:
model_training.training(
    model=efficientnet_v2_s_model, 
    optimizer=optimizer, 
    loss_fn=loss, 
    train_dataloader=train_dataloader, 
    val_dataloader=val_dataloader, 
    model_path="models/EfficientNetV2", 
    start_epoch=4,
    epochs=500,
    early_stop_thresh=50,
)

End of epoch 5: training accuracy = 97.75%, training loss = 0.07468436123597626, training time taken = 881.31 seconds
End of epoch 5: validation accuracy = 98.56%, validation loss = 0.045008501832766314, validation time taken = 300.58 seconds
End of epoch 6: training accuracy = 98.16%, training loss = 0.06048193436551608, training time taken = 1618.39 seconds
End of epoch 6: validation accuracy = 98.96%, validation loss = 0.03279907385617026, validation time taken = 284.90 seconds
End of epoch 7: training accuracy = 98.60%, training loss = 0.04486010109755495, training time taken = 1456.32 seconds
End of epoch 7: validation accuracy = 98.22%, validation loss = 0.05949891274853529, validation time taken = 246.57 seconds
End of epoch 8: training accuracy = 98.71%, training loss = 0.04237468255008827, training time taken = 1494.08 seconds
End of epoch 8: validation accuracy = 99.10%, validation loss = 0.030668059088461225, validation time taken = 254.65 seconds
End of epoch 9: training ac

KeyboardInterrupt: 

In [11]:
torch.save(efficientnet_v2_s_model, 'models/EfficientNetV2/efficientnet_v2_s_model_epoch_10.pth')

In [ ]:
import preprocessing_func_2

df_big_long_wav = pd.read_csv('data/metadata.csv')
df_val_long_wav = df_big_long_wav[df_big_long_wav["subset"]=="validation"]

def get_df_from_class(class_num, df):
    new_df = df[df["label"] == class_num]
    return new_df

for i in range(66):
    df_val_new = get_df_from_class(i, df_val_long_wav)
    paths, labels = list(df_val_new["path"]), list(df_val_new["label"])
    non_normalised_generator = preprocessing_func_2.non_normalised_data_generator(paths, labels)
    normalised_generator = preprocessing_func_2.normalised_data_generator(
        non_normalised_generator, mean, std)
    print(f"we are now in class {i}")
    final_pred = model_eval.evaluation(efficientnet_v2_s_model, normalised_generator)
    print()

In [ ]:
df_big_long_wav = pd.read_csv('data/metadata.csv')
df_val_long_wav = df_big_long_wav[df_big_long_wav["subset"]=="validation"]


paths, labels = list(df_val_long_wav["path"]), list(df_val_long_wav["label"])
non_normalised_generator = preprocessing_func_2.non_normalised_data_generator(paths, labels)
normalised_generator = preprocessing_func_2.normalised_data_generator(non_normalised_generator, mean, std)
model_eval.evaluation(efficientnet_v2_s_model, normalised_generator, loss)